<!-- NOTEBOOK_METADATA source: "⚠️ Jupyter Notebook" title: "Observability for Claude Agent SDK with Langfuse" sidebarTitle: "Claude Agent SDK" logo: "/images/integrations/claude_icon.png" description: "Learn how to trace Claude Agent SDK applications with Langfuse via OpenTelemetry" category: "Integrations" -->

# Integrate Langfuse with Claude Agent SDK

This notebook demonstrates how to capture detailed traces from the [Claude Agent SDK](https://github.com/anthropics/claude-agent-sdk-python) with **[Langfuse](https://langfuse.com)** using OpenTelemetry.

> **What is Claude Agent SDK?**  
> The [Claude Agent SDK](https://platform.claude.com/docs/en/agent-sdk/overview) is Anthropic's open-source framework for building AI agents. It provides a clean API for creating tool-using agents, including native support for MCP.

> **What is Langfuse?**  
> [Langfuse](https://langfuse.com) is the open source LLM engineering platform. It provides detailed tracing, monitoring, and analytics for every prompt, model response, and tool call in your agent, making it easy to debug, evaluate, and iterate on LLM applications.

<!-- STEPS_START -->
## Step 1: Install Dependencies

In [ ]:
%pip install langfuse claude-agent-sdk "langsmith[claude-agent-sdk]" "langsmith[otel]" -q

## Step 2: Set Up Environment Variables

Set up your **Langfuse** API keys ([Langfuse Cloud](https://cloud.langfuse.com) or [self-hosted](https://langfuse.com/self-hosting)) and your **Anthropic** API key ([Anthropic Console](https://console.anthropic.com/)).

In [ ]:
import os

# Get keys for your project from the project settings page: https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..."
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..."
os.environ["LANGFUSE_BASE_URL"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_BASE_URL"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

# Your Anthropic API key
os.environ["ANTHROPIC_API_KEY"] = "sk-ant-..."

# Configure LangChain OpenTelemetry instrumentation for Claude Agent SDK
os.environ["LANGSMITH_OTEL_ENABLED"] = "true"
os.environ["LANGSMITH_OTEL_ONLY"] = "true"
os.environ["LANGSMITH_TRACING"] = "true"

With the environment variables set, we can now initialize the Langfuse client. `get_client()` initializes the Langfuse client using the credentials provided in the environment variables.

In [ ]:
from langfuse import get_client

langfuse = get_client()

# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

## Step 3: OpenTelemetry Instrumentation

Use [LangChain's Claude Agent SDK instrumentation library](https://github.com/langchain-ai/langsmith-sdk/tree/main/python/langsmith/integrations/claude_agent_sdk) to instrument the agent SDK and send OpenTelemetry spans to Langfuse.

In [ ]:
from langsmith.integrations.claude_agent_sdk import configure_claude_agent_sdk

# Configure Claude Agent SDK with OpenTelemetry tracing
configure_claude_agent_sdk()

## Step 4: Build a Hello World Agent

Every tool call and model completion is captured as an OpenTelemetry span and forwarded to Langfuse.

In [ ]:
import asyncio
from claude_agent_sdk import (
    ClaudeAgentOptions,
    ClaudeSDKClient,
    tool,
    create_sdk_mcp_server,
)
from typing import Any

@tool(
    "get_weather",
    "Gets the current weather for a given city",
    {
        "city": str,
    },
)
async def get_weather(args: dict[str, Any]) -> dict[str, Any]:
    """Simulated weather lookup tool"""
    city = args["city"]

    # Simulated weather data
    weather_data = {
        "Berlin": "Cloudy, 59°F",
        "New York": "Sunny, 75°F",
    }

    weather = weather_data.get(city, "Weather data not available")
    return {"content": [{"type": "text", "text": f"Weather in {city}: {weather}"}]}


async def main():
    # Create SDK MCP server with the weather tool
    weather_server = create_sdk_mcp_server(
        name="weather",
        version="1.0.0",
        tools=[get_weather],
    )

    options = ClaudeAgentOptions(
        model="claude-sonnet-4-5-20250929",
        system_prompt="You are a friendly travel assistant who helps with weather information.",
        mcp_servers={"weather": weather_server},
        allowed_tools=["mcp__weather__get_weather"],
    )

    async with ClaudeSDKClient(options=options) as client:
        await client.query("What's the weather like in Berlin and New York?")

        async for message in client.receive_response():
            print(message)


await main()

## Step 5: View the Trace in Langfuse

Head over to your **Langfuse dashboard → Traces**. You should see traces including all tool calls and model inputs/outputs.

![Claude Agent SDK example trace in Langfuse](https://langfuse.com/images/cookbook/integration_claude_agent_sdk/claude-agent-sdk-example-trace.png)

[Link to trace in Langfuse](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/097f1d8982fa909b8cffb14a166ec272?timestamp=2025-12-22T10:46:15.528Z)

<!-- STEPS_END -->

<!-- MARKDOWN_COMPONENT name: "LearnMore" path: "@/components-mdx/integration-learn-more.mdx" -->